# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] <urlopen error [WinError 10060] Ein Verbindungsversuch ist
[nltk_data]     fehlgeschlagen, da die Gegenstelle nach einer
[nltk_data]     bestimmten Zeitspanne nicht richtig reagiert hat, oder
[nltk_data]     die hergestellte Verbindung war fehlerhaft, da der
[nltk_data]     verbundene Host nicht reagiert hat>


False

In [10]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from sqlalchemy import create_engine

In [11]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('select * from InsertTableName',engine)
X = df.message.values
Y = df.drop(['id','message','original','genre'], 1).values


In [8]:
set(df.columns.values)-set(['id','message','original','genre'])

{'aid_centers',
 'aid_related',
 'buildings',
 'child_alone',
 'clothing',
 'cold',
 'death',
 'direct_report',
 'earthquake',
 'electricity',
 'fire',
 'floods',
 'food',
 'hospitals',
 'infrastructure_related',
 'medical_help',
 'medical_products',
 'military',
 'missing_people',
 'money',
 'offer',
 'other_aid',
 'other_infrastructure',
 'other_weather',
 'refugees',
 'related',
 'request',
 'search_and_rescue',
 'security',
 'shelter',
 'shops',
 'storm',
 'tools',
 'transport',
 'water',
 'weather_related'}

### 2. Write a tokenization function to process your text data

In [4]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mic', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
col_names = df.drop(['id','message','original','genre'], 1).columns.values
len(col_names)
y_pred_df = pd.DataFrame(y_pred, columns=col_names)
y_test_df = pd.DataFrame(y_test, columns=col_names)

col_names[0]

'related'

In [9]:
y_pred_df['related'].nunique()

Y_df = pd.DataFrame(Y, columns=col_names)
Y_df.query('related == 2')['related']

117      2
219      2
305      2
460      2
576      2
        ..
20324    2
20495    2
22326    2
23382    2
25215    2
Name: related, Length: 188, dtype: int64

In [10]:
def display_results(y_test, y_pred):
    for col in y_pred.columns:
        
        print(classification_report(y_test[col].values,y_pred[col].values))
   
        

In [11]:
display_results(y_test_df, y_pred_df)

              precision    recall  f1-score   support

           0       0.64      0.50      0.56      1466
           1       0.86      0.91      0.89      5041
           2       0.29      0.40      0.34        47

   micro avg       0.82      0.82      0.82      6554
   macro avg       0.59      0.60      0.59      6554
weighted avg       0.81      0.82      0.81      6554

              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5433
           1       0.79      0.44      0.57      1121

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.84      0.71      0.75      6554
weighted avg       0.88      0.88      0.87      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6181
           1       0.50      0.04      0.08       373

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.72      0.52      0.52      6554
weighted avg       0.92      0.94      0.92      6554

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      5319
           1       0.74      0.33      0.45      1235

   micro avg       0.85      0.85      0.85      6554
   macro avg       0.80      0.65      0.68      6554
weighted avg       0.84      0.85      0.83      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
list(pipeline.get_params().keys())

['memory',
 'steps',
 'vect',
 'tfidf',
 'mic',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'mic__estimator__bootstrap',
 'mic__estimator__class_weight',
 'mic__estimator__criterion',
 'mic__estimator__max_depth',
 'mic__estimator__max_features',
 'mic__estimator__max_leaf_nodes',
 'mic__estimator__min_impurity_decrease',
 'mic__estimator__min_impurity_split',
 'mic__estimator__min_samples_leaf',
 'mic__estimator__min_samples_split',
 'mic__estimator__min_weight_fraction_leaf',
 'mic__estimator__n_estimators',
 'mic__estimator__n_jobs',
 'mic__estimator__oob_score',
 'mic__estimator__random_state',
 'mic__estimat

In [13]:
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'mic__estimator__n_estimators':  [10, 50],
    #'mic__estimator__min_samples_split': [2, 3, 4]
}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
import pickle
pickle.dump(cv, open("model.pickle", "wb"))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.